**Project desription**
I work with a table of contexts in language X and their translations into Russian. I study the argument structure of some verbs, what is why I need a tool which will help me to find all lines with particular verb (in any of its forms) quickly. I decided to lemmatize the translation column and add that output as a new column to the same table.

My code:
1. Opens a csv file (table with a column of Russian translations) 
2. Chooses the column of contexts in Russian (comment: some contexts occur more than once - it is just fine, these rows differ in other columns of datatable)
3. Lemmatizes that colomn (without stopwords as this is not needed here), creates a list
4. Saves the lemmatized list as a text
5.  takes the original contexts and lemmatized contexts in dictionary, which is transformed then in a new csv 
5.  was supposed to combine old csv with new csv, but gives character mistake...again... I put encoding everywhere 'utf8', but it doesn't help:)))

I also tried to 
*. Add the result list as a new column to the original table 
-  but it gives me an error
#charmap' codec can't encode character '\xfd' in position 13701: character maps to <undefined>.

**0.1.** Openning packages 

In [1]:

! pip install pandas
import pandas as pd
import csv as csv
import sqlite3

In [2]:
import re
import gensim
import logging
import nltk.data
import urllib.request
from bs4 import BeautifulSoup
from nltk.corpus import stopwords
from gensim.models import word2vec

import warnings
warnings.filterwarnings('ignore')

import tqdm
import pymystem3

In [3]:
from string import punctuation

**0.2.** Preparation for lemmatization

In [4]:
from tqdm import tqdm
from pymystem3 import Mystem
import nltk
nltk.download('stopwords')
m = Mystem()
#sw = stopwords.words('russian') #as long as here I need all verbs, including COP

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Sarah\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


**1.** and **2.** Openning CSV and extracting a column as a list of strings

In [6]:
#this code gives me a list of contexts which are to be lemmatized
with open('Botlikh_Ru.csv', 'r', encoding='utf8') as ff:
    csv_reader = csv.reader(ff, delimiter=',')
    for lines in csv_reader:
        #print(lines[1]) #works, shows sentences I need
        trans = list(zip(*csv_reader)) # gives list with four tuple objects - columns of table
        #print (trans)
        trans_context = trans[1] #tuple with needed column (contexts which are in column 2 of csv)
        #print(trans_context)
        trans_context_list = list(trans_context) #convert tuple into list
        #print(trans_context_list) #######as a list of strings
        
#turns the list into one string - was not used
t_c_l_one = '\n'.join(trans_context_list)
print(t_c_l_one) #########as one string 

выбраться или выйти в люди
выбраться или выйти в люди
вздыбиться
вздор
не говори вздор
вкатить бочку в сарай
подозвать
подогнать к (противоположной) стене
подогнать к (противоположной) стене
вблизи; поближе
она подошла к этой корове
с близкого расстояния 2. мимо
ближайший
загнать скот в хлев
хлев залило водой
подлетать (близко)
замешаться; оказаться причастным к чему-л.
вмешаться; вмешиваться; совать нос куда-л.; 
лезть не в своё дело
вмешаться в спор
вмешаться в спор
вмешаться в драку
не вмешивайся в мои дела
нелюдимый
нелюдимый
не вмешивайся не в свои дела; не суй свой нос в чужие дела
а) разнять дерущихся; б) принимать участие; 
привлечь; вовлечь; замешать; побудить принять участие в чём-л.; 
вмешиваться; лезть в чужие дела
ввести в какое-л. общество
вмешивать; заставлять участвовать против воли
подпустить
подпуск
приблизиться
приобщить; дать возможность приобщить;
подослать
вылазка
оборваться; упасть куда-л
влопаться в беду
влопаться в беду
влопаться; попасть в неприятное положение

**3.** Lemmatizing the column of translations

In [10]:
new_lines = []
for line in tqdm(trans_context_list):
    line = ' '.join([w for w in line.split() if w.isalpha()])
    newline = ''.join(m.lemmatize(line))
    new_lines.append(newline)

100%|███████████████████████████████████████████████████████████████████████████| 3306/3306 [11:08:42<00:00,  1.96s/it]


In [56]:
new_lines[0:10]

['выбираться или выходить в человек\n',
 'выбираться или выходить в человек\n',
 'вздыбиться\n',
 'вздор\n',
 'не говорить вздор\n',
 'вкатить бочка в сарай\n',
 'подзывать\n',
 'подгонять к стена\n',
 'подгонять к стена\n',
 'близко\n']

**4.** Saving the lemmatized data into a text file

In [59]:
#saving the lemmatized data into a text file       
with open('t_c_lem.txt', 'w', encoding='utf8') as f1:
    for line in new_lines:
        f1.write(line)
        
#I had to save it in text, as it didn't write it to a csv: 
#error
#charmap' codec can't encode character '\xfd' in position 13701: character maps to <undefined>

In [63]:
test_nl = new_lines #how many elements are there in lemmatized list?
len(test_nl)

3306

In [68]:
 len(trans_context_list) #how many elements are there in original list of contexts?

3306

**5.1.** Combining contexts from initial table with lemmatized contexts in one csv

In [77]:
#combining the contexts with lemmatized texts into one dictionary

ke = trans_context_list #the original contexts
va = test_nl #the lemmatized contexts

df = pd.DataFrame({'Context' : ke, 'Context_Lemma' : va}) 

In [73]:
#saving dictionary into csv

df.to_csv('Botl_2.csv', index=False, encoding='utf-8')

**5.2.** a try to combine the original csv with the csv with lemmatized contexts based on context table -  doesn't work

In [80]:
# a try to combine the original csv with the csv with lemmatized contexts based on context table -  doesn't work

#openning the 1st table
import sqlite3
import csv
con = sqlite3.connect("botlikh_novv.db")
cur = con.cursor()
cur.execute("DROP TABLE IF EXISTS botlikh_novv")
cur.execute("CREATE TABLE botlikh_novv (ID, Translation, Host_Lemma, Host_Trans, Form, Context);")  # all the columns names

with open('Botlikh_novv.csv','r', encoding='utf-8') as fin:
    dr = csv.DictReader(fin)  # a comma is the default separator
    to_db = [(i['ID'], i['Translation'], i['Host_Lemma'], i['Host_Trans'], i['Form'], i['Context']) for i in dr]  # put the columns' names again

cur.executemany("INSERT INTO botlikh_novv (ID, Translation, Host_Lemma, Host_Trans, Form, Context) VALUES (?, ?, ?, ?, ?, ?);", to_db)  # and again, the names of the columns

for i in cur.execute("SELECT * FROM botlikh_novv"):
    print(i)
    
con.commit()
#con.close()


#openning the second table
import sqlite3
import csv
conl = sqlite3.connect("botlikh_novb.db")
curl = con.cursor()
curl.execute("DROP TABLE IF EXISTS botlikh_novb")
curl.execute("CREATE TABLE botlikh_novb (Context, Context_Lemma);")  # all the columns names

with open('Botl_2.csv','r', encoding='utf-8') as finl:
    drl = csv.DictReader(finl)  # a comma is the default separator
    to_dbl = [(i['Context'], i['Context_Lemma']) for i in drl]  # put the columns' names again

curl.executemany("INSERT INTO botlikh_novb (Context, Context_Lemma) VALUES (?, ?);", to_dbl)  # and again, the names of the columns

for il in curl.execute("SELECT * FROM botlikh_novb"):
    print(il)
    
conl.commit()
#conl.close()


#Combining the tables
cur.execute ("SELECT * FROM botlikh_novb LEFT JOIN botlikh_novv ON botlikh_novb.Context = botlikh_novv.Context")

UnicodeEncodeError: 'charmap' codec can't encode character '\u0301' in position 40: character maps to <undefined>